Returns: https://www.dillards.com/c/returns
- Purchases outside these dates will follow our standard 30 day return policy.
- Bring any merchandise eligible for return to a Dillard's location near you (excluding clearance centers) along with your receipt or proof of purchase.


Refund group
Add constraint within 30 days
How to deal with duplicates?
Multiple purchases returns
- Total: 2751811
- Focus on item, so drop all duplicates: 2183980
- Group and count number of items being purchased and returned: 2163109


How to merge back to transact?

Products that are not return do not have datediff

Use union?

Then merge with other tables:
- TRNSACT: quantity, price, discount, discount_flag, datediff, weekend_flag, holiday_flag
- STATE?
- STRINFO: state
- SKUINFO: color, size?
- DEPTINFO: dept (dummmy or group based on price)


- Logistics Regression
- Random Forest
- SVM
- Boosting

https://apps.bea.gov/regional/histdata/releases/0907gmp/index.cfm

https://www.census.gov/data/tables/2004/demo/popproj/2004-int-popproj-summary.html

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import holidays
from datetime import datetime

In [3]:
df = pd.read_csv("test.csv")
df 

,SKU,SALEDATE,INTERID,STYPE,QUANTITY,ORGPRICE,AMT,CITY,STATE,COST,RETAIL,DEPT,COLOR,SIZE,BRAND,ZIP
0,1000007,2005-08-05,744804315,P,1,16.0,16.00,PHOENIX,AZ,7.0,16.00,6404.0,LT. BLUE,L,BILLABON,85032
1,1000007,2005-08-09,478109057,P,1,16.0,16.00,CHANDLER,AZ,7.0,16.00,6404.0,LT. BLUE,L,BILLABON,85226
2,1000007,2005-08-09,554308032,P,1,16.0,16.00,SANDY,UT,7.0,16.00,6404.0,LT. BLUE,L,BILLABON,84070
3,1000007,2005-08-11,141107309,P,1,16.0,16.00,PHOENIX,AZ,7.0,16.00,6404.0,LT. BLUE,L,BILLABON,85032
4,1000007,2005-08-19,56509933,P,1,16.0,16.00,EL CENTRO,CA,7.0,16.00,6404.0,LT. BLUE,L,BILLABON,92243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22266266,9999997,2005-04-24,726601646,P,1,39.0,39.00,AUSTIN,TX,15.0,19.50,2503.0,210CHOPNK,7,RAMPAGE,78746
22266267,9999997,2005-05-05,854805967,P,1,39.0,39.00,SAN ANTONIO,TX,15.0,19.50,2503.0,210CHOPNK,7,RAMPAGE,78216
22266268,9999997,2005-06-08,384909302,R,1,39.0,39.00,ARLINGTON,TX,15.0,9.75,2503.0,210CHOPNK,7,RAMPAGE,76015
22266269,9999997,2005-06-09,47603838,P,1,39.0,31.20,ARLINGTON,TX,15.0,9.75,2503.0,210CHOPNK,7,RAMPAGE,76015


In [5]:
df["STYPE"].value_counts()

P    21164433
R     1101838
Name: STYPE, dtype: int64

In [10]:
df["discount"] = df["AMT"] / (df["ORGPRICE"] * df["QUANTITY"])
df["discount_flag"] = (df["discount"] < 1).astype(int)

There are infinite values. Can we set as a number? 

In [17]:
us_holidays = holidays.US()
df['holiday_flag'] = df['SALEDATE'].isin(us_holidays).astype(int)
df

,SKU,SALEDATE,INTERID,STYPE,QUANTITY,ORGPRICE,AMT,CITY,STATE,COST,RETAIL,DEPT,COLOR,SIZE,BRAND,ZIP,discount,discount_flag,holiday_flag
0,1000007,2005-08-05,744804315,P,1,16.0,16.00,PHOENIX,AZ,7.0,16.00,6404.0,LT. BLUE,L,BILLABON,85032,1.00,0,0
1,1000007,2005-08-09,478109057,P,1,16.0,16.00,CHANDLER,AZ,7.0,16.00,6404.0,LT. BLUE,L,BILLABON,85226,1.00,0,0
2,1000007,2005-08-09,554308032,P,1,16.0,16.00,SANDY,UT,7.0,16.00,6404.0,LT. BLUE,L,BILLABON,84070,1.00,0,0
3,1000007,2005-08-11,141107309,P,1,16.0,16.00,PHOENIX,AZ,7.0,16.00,6404.0,LT. BLUE,L,BILLABON,85032,1.00,0,0
4,1000007,2005-08-19,56509933,P,1,16.0,16.00,EL CENTRO,CA,7.0,16.00,6404.0,LT. BLUE,L,BILLABON,92243,1.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22266266,9999997,2005-04-24,726601646,P,1,39.0,39.00,AUSTIN,TX,15.0,19.50,2503.0,210CHOPNK,7,RAMPAGE,78746,1.00,0,0
22266267,9999997,2005-05-05,854805967,P,1,39.0,39.00,SAN ANTONIO,TX,15.0,19.50,2503.0,210CHOPNK,7,RAMPAGE,78216,1.00,0,0
22266268,9999997,2005-06-08,384909302,R,1,39.0,39.00,ARLINGTON,TX,15.0,9.75,2503.0,210CHOPNK,7,RAMPAGE,76015,1.00,0,0
22266269,9999997,2005-06-09,47603838,P,1,39.0,31.20,ARLINGTON,TX,15.0,9.75,2503.0,210CHOPNK,7,RAMPAGE,76015,0.80,1,0


In [24]:
df['SALEDATE'] = pd.to_datetime(df['SALEDATE'])
df['weekend_flag'] = (df['SALEDATE'].dt.weekday.astype(int) >= 5).astype(int)
df

,SKU,SALEDATE,INTERID,STYPE,QUANTITY,ORGPRICE,AMT,CITY,STATE,COST,RETAIL,DEPT,COLOR,SIZE,BRAND,ZIP,discount,discount_flag,holiday_flag,weekend_flag
0,1000007,2005-08-05,744804315,P,1,16.0,16.00,PHOENIX,AZ,7.0,16.00,6404.0,LT. BLUE,L,BILLABON,85032,1.00,0,0,0
1,1000007,2005-08-09,478109057,P,1,16.0,16.00,CHANDLER,AZ,7.0,16.00,6404.0,LT. BLUE,L,BILLABON,85226,1.00,0,0,0
2,1000007,2005-08-09,554308032,P,1,16.0,16.00,SANDY,UT,7.0,16.00,6404.0,LT. BLUE,L,BILLABON,84070,1.00,0,0,0
3,1000007,2005-08-11,141107309,P,1,16.0,16.00,PHOENIX,AZ,7.0,16.00,6404.0,LT. BLUE,L,BILLABON,85032,1.00,0,0,0
4,1000007,2005-08-19,56509933,P,1,16.0,16.00,EL CENTRO,CA,7.0,16.00,6404.0,LT. BLUE,L,BILLABON,92243,1.00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22266266,9999997,2005-04-24,726601646,P,1,39.0,39.00,AUSTIN,TX,15.0,19.50,2503.0,210CHOPNK,7,RAMPAGE,78746,1.00,0,0,1
22266267,9999997,2005-05-05,854805967,P,1,39.0,39.00,SAN ANTONIO,TX,15.0,19.50,2503.0,210CHOPNK,7,RAMPAGE,78216,1.00,0,0,0
22266268,9999997,2005-06-08,384909302,R,1,39.0,39.00,ARLINGTON,TX,15.0,9.75,2503.0,210CHOPNK,7,RAMPAGE,76015,1.00,0,0,0
22266269,9999997,2005-06-09,47603838,P,1,39.0,31.20,ARLINGTON,TX,15.0,9.75,2503.0,210CHOPNK,7,RAMPAGE,76015,0.80,1,0,0


In [25]:
color_dict = {
    'black': ['black', 'blk', 'onyx', 'obsidian'],
    'blue': ['blue', 'navy', 'indigo', 'denim', 'royal', 'nav', 'midnight', 'newport', 'aqua', 'turquoise', 'turq', 'teal'], 
    'red': ['red', 'burgundy', 'cardinals', 'wine', 'maroon', 'cardinal', 'ruby'],
    'white': ['white', 'cream', 'ivory', 'wht', 'wh', 'vanilla'],
    'pink': ['pink', 'rose', 'fuchsia', 'fuschia', 'berry'],
    'brown': ['brown', 'tan', 'khaki', 'chocolate', 'camel', 'beige', 'espresso', 'bronze', 'natural', 'taupe'],
    'grey': ['grey', 'silver', 'charcoal', 'stone'],
    'yellow': ['yellow', 'gold', 'champagne', 'sand'],
    'green': ['green', 'olive', 'lime', 'sage'],
    'orange': ['orange', 'coral'],
    'purple': ['purple', 'lilac', 'plum', 'lavender'],
    'multi': ['multi', 'assorted', 'asst']
}

In [26]:
def replace_color(text):
    for group, lst in color_dict.items():
        for word in lst:
            if word in text.lower():
                return group
    return "other"

In [32]:
df["COLOR"] = df["COLOR"].astype(str)
df["color_group"] = df["COLOR"].apply(replace_color)

In [33]:
df["color_group"].value_counts()

other     9704083
black     2890005
white     2562895
blue      1722427
brown     1083950
pink      1022125
multi      661324
red        603033
yellow     516538
grey       461471
green      451402
orange     389428
purple     197590
Name: color_group, dtype: int64

In [40]:
size_dict = {
    'large': ["L", "XL", "XXL", "XXXL", "4X", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "22", "24", "25", "26", "27", "28", "29", "30", "31", "32", "34", 
              "35", "36", "37", "38", "1X", "2X", "3X", "6X", "130M", "120M", "115M", "110M", "105M", "100M", "095M","100N", "095N", "090M", "090N", "090W", "XLT", "2XT", "3XT", "4XT", 
              "16W", "18W", "20W", "22W", "24W", "L    ND", "XL   ND", "XXL  ND", "LT", "2T", "3T", "LARGE"] ,
    'small': ["M", "S", "XS", "XXS", "00", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "085M", "080M", "075M", "070M", "065M", "060M", "055M", "050M", "085N", "080N", 
              "075N", "070N", "085W", "080W", "075W", "070W", "M    ND", "S    ND"],
    "one": ["ALL", "ONE", "NO SIZE"]
}

In [41]:
def replace_size(text):
    for group, lst in size_dict.items():
        for word in lst:
            if word == text:
                return group
    return "other"

In [42]:
df["SIZE"] = df["SIZE"].str.strip()
df["size_group"] = df["SIZE"].apply(replace_size)

In [43]:
df["size_group"].value_counts()

other    7871421
large    5696716
small    5323624
one      3374510
Name: size_group, dtype: int64

In [45]:
df.head()

,SKU,SALEDATE,INTERID,STYPE,QUANTITY,ORGPRICE,AMT,CITY,STATE,COST,...,COLOR,SIZE,BRAND,ZIP,discount,discount_flag,holiday_flag,weekend_flag,color_group,size_group
0,1000007,2005-08-05,744804315,P,1,16.0,16.0,PHOENIX,AZ,7.0,...,LT. BLUE,L,BILLABON,85032,1.0,0,0,0,blue,large
1,1000007,2005-08-09,478109057,P,1,16.0,16.0,CHANDLER,AZ,7.0,...,LT. BLUE,L,BILLABON,85226,1.0,0,0,0,blue,large
2,1000007,2005-08-09,554308032,P,1,16.0,16.0,SANDY,UT,7.0,...,LT. BLUE,L,BILLABON,84070,1.0,0,0,0,blue,large
3,1000007,2005-08-11,141107309,P,1,16.0,16.0,PHOENIX,AZ,7.0,...,LT. BLUE,L,BILLABON,85032,1.0,0,0,0,blue,large
4,1000007,2005-08-19,56509933,P,1,16.0,16.0,EL CENTRO,CA,7.0,...,LT. BLUE,L,BILLABON,92243,1.0,0,0,0,blue,large


- Brand
- Incorporate economic factors